Recordemos:

$\mbox{[MIN]} c^T x$

subject to:

$Ax=b$

Dividimos en básicas y no básicas

$c_B x_B + c_N x_N$

sujeto a:

$B x_B + N x_N = b$

operando las restricciones: $x_B = B^{-1} b - B^{-1} N x_N$

y sustituyendo en la función objetivo: $C_B (B^{-1} b - B^{-1} N x_N) + c_N x_N$

Por lo que el modelo se puede escribir como:

$\mbox{[MAX]}~c_B B^{-1} b + (c_N - c_B B^{-1} N) x_N$

sujeto a:

$x_B + B^{-1} N x_N = B^{-1} b$

Entrará la de mayor coste reducido (maximo cRed)
Saldrá la de coeficiente

In [ ]:
#introducimos datos
#Se asume que el PL es de MINIMIZACIÓN y que las restricciones están de la forma Ax=b
c=[-2500 -4000 0 0 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]

In [ ]:
#Ajuste de data
function b_mayor_cero(A,b)
    for i=1:length(b)
        if b[i]<0
            b[i]=-b[i]
            A[i,:]=-A[i,:]
        end
    end
    return A,b
end

#Fase 1
#Construcción del problema auxiliar
function auxproblem(A,b,c)
    cf1=hcat(zeros(length(c))',ones(size(A,1))');
    I=eye(size(A,1),size(A,1));
    A=hcat(A,I);
        
    basicas=Vector(size(A,2)-size(I,1)+1:size(A,2));
    nobasicas=Vector(1:size(A,2)-size(I,1));
    B=A[:,size(A,2)-size(I,1)+1:size(A,2)];
    cB=cf1[size(A,2)-size(I,1)+1:size(A,2)]';
    N=A[:,1:size(A,2)-size(I,1)];
    cN=cf1[1:size(A,2)-size(I,1)]';
    
    return basicas,nobasicas,B,N,cB,cN
end    

In [ ]:
#seleccionar variable que entra y sale
function selectPivote(costosBasicos,matrizN,rhs,Binv)
    #println("matrizN: ",matrizN)
    #println("rhs: ",rhs)
    println("costosBasicos: ",costosBasicos)
    opt=false
    noacotado=false
    #devuelve posicion con costo reducido negativo
    minCol=0
    for i in 1:length(costosBasicos)
        if costosBasicos[i]<0
            minCol=i
            break
        end
    end
    #println("Entrante: ", minCol)
    #Validación optimalidad
    if minCol==0
        println("Solucion Optima")
        opt=true
        return -1,-1,opt,noacotado
    end
    
    #Implementción regla lexicográfica
    u=matrizN[:,minCol]
    if all(u.<=0)
        noacotado=true
        println("El PL planteado es no acotado")
        return -1,-1,opt,noacotado
    end
    d=-rhs./u
    dm=d[(d.<=0)&(u.>0)]
    if all(dm.==0)==true
        newCol=1
    else
        teta=maximum(dm)
        teta_pos=find(x->x==teta,d)
        allv=vcat(basicas,nobasicas)
        A=hcat(Binv,matrizN)
        orden=sortperm(allv)
        A=A[:,orden]
        contador=1
        while length(teta_pos)>1
            if (all(A[teta_pos,contador].==A[teta_pos,contador][1]))
                contador=contador+1
            else
                delete=maximum(A[teta_pos,contador])
                delwhere=find(x->x.==delete,A[teta_pos,contador])
                teta_pos=teta_pos[setdiff(1:length(teta_pos),delwhere)]
            end
        end
        newCol=teta_pos[1]
    end
    println("entra columna ",minCol," sale ",newCol)
    minCol,newCol,opt,noacotado
end

#realizar operaciones de una iteración del simplex
function iteracionSimplex(B,cB,N,cN,b,basicas,nobasicas,fase)
    #iteración del simplex
    Binv=B^-1
    obj=cB*Binv*b
    println("obj: ",obj)
    cRed=cN-cB*Binv*N
    #println("costos reducidos:",cRed)
    sol=Binv*b
    #println("solucion:",sol)
    Nprima=Binv*N
    #println("Nprima:",Nprima)
    if fase==1 && obj[1]==0
        return obj,-1,-1, true, false
    else
        v1,v2,opt,noacotado = selectPivote(cRed,Nprima,sol,Binv)
        return obj,v1,v2, opt, noacotado
    end
end

In [ ]:
#construimos base inicial
basicas = Array{Int16}()
nobasicas = Array{Int16}()
#vamos a dimensionar valores de calculo
B=Array{Float64}()
cB=Array{Float64}()
N=Array{Float64}()
cN=Array{Float64}()

#inicializamos B, cB, N, cN
A,b=b_mayor_cero(A,b);
basicas,nobasicas,B,N,cB,cN=auxproblem(A,b,c)

In [ ]:
function ordenit(B,cB,N,cN,b,basicas,nobasicas,fase)
    while true
        obj,variableIn,variableOut,opt, noacotado = iteracionSimplex(B,cB,N,cN,b,basicas,nobasicas,fase)
        if variableIn != -1
            #Ordenando vectores y matrices de acuerdo a sus subindices de menor a mayor
            basicas[variableOut] , nobasicas[variableIn] = nobasicas[variableIn] , basicas[variableOut];
            cB[1,variableOut] , cN[1,variableIn] = cN[1,variableIn] , cB[1,variableOut]; 
            B[:, variableOut],  N[:,variableIn] = N[:,variableIn] , B[:,variableOut];
            ordb=sortperm(basicas);
            ordnb=sortperm(nobasicas);
            cB=cB[ordb]';
            cN=cN[ordnb]';
            B=B[:,ordb];
            N=N[:,ordnb];
            basicas=basicas[ordb];
            nobasicas=nobasicas[ordnb];
            println("obj: ",obj[1]," cB: ", cB, " cN: ",cN," basicas: ",basicas," noBasicas ",nobasicas)
            println("##########---#########")
            z=obj[1]
        else
            if noacotado==false
                println("obj: ",obj[1]," basicas: ",basicas," valores ",B^(-1)*b)
                println("##########---#########")
                z=obj[1]
                return z,B,N,basicas,nobasicas,cB,cN
            else
                println("obj: ", -Inf, "El problema es no acotado")
            end
        end
    end
end

In [ ]:
#Box para iterar en primera fase
z,B,N,basicas,nobasicas,cB,cN=ordenit(B,cB,N,cN,b,basicas,nobasicas,1)

In [ ]:
if z==0
    while length(find(x->x.>size(A,2),basicas))>0
        newCol=0
        for i in length(basicas):-1:1
            if basicas[i]>size(A,2)
                newCol=i
                break
            end
        end
        
        if all(x->x.==0,(B^-1*A)[newCol,:])
            #En este if se eliminan filas que sean LI y se resuelve de nuevo el problema
            println("La restricción ", newCol," es LI con respecto a una(s) de las existentes, pór lo tanto será eliminada y se resolverá de nuevo el problema")
            A=A[setdiff(1:size(A,1),basicas[newCol]-size(A,2)),:];
            b=b[setdiff(1:size(b,1),basicas[newCol]-size(A,2)),:];
            basicas = Array{Int16}();
            nobasicas = Array{Int16}();
            B=Array{Float64}()
            cB=Array{Float64}()
            N=Array{Float64}()
            cN=Array{Float64}()

            #inicializamos B, cB, N, cN
            basicas,nobasicas,B,N,cB,cN=auxproblem(A,b,c);
            
            #reconstruyendo la solución
            z,B,N,basicas,nobasicas,cB,cN=ordenit(B,cB,N,cN,b,basicas,nobasicas,1)
        else
            #Esta sección ayuda a eliminar una variable artificial de fase 1 que quedó en la base óptima de dicho problema
            println("hay que sacar variables artificiales de la base")
            minCol=0
            for i in 1:length(nobasicas)
                if nobasicas[i]<=size(A,2)
                    if (B^-1*A)[newCol,nobasicas[i]]!=0
                        #println(B[:,setdiff(1:size(B,2),newCol)],(B^-1*A)[:,nobasicas[i]])
                        if det(hcat(B[:,setdiff(1:size(B,2),newCol)],(B^-1*A)[:,nobasicas[i]]))!=0
                            minCol=i
                            break
                        end
                    end
                end
            end
            println("Sale la variable de la columna: ",newCol," e ingresa la variable de la columna: ",minCol)
            basicas[newCol] , nobasicas[minCol] = nobasicas[minCol] , basicas[newCol];
            cB[1,newCol] , cN[1,newCol] = cN[1,newCol] , cB[1,newCol]; 
            B[:, newCol],  N[:,newCol] = N[:,newCol] , B[:,newCol];
            ordb=sortperm(basicas);
            ordnb=sortperm(nobasicas);
            cB=cB[ordb]';
            cN=cN[ordnb]';
            B=B[:,ordb];
            N=N[:,ordnb];
            basicas=basicas[ordb];
            nobasicas=nobasicas[ordnb];    
        end
    end    
    nobasicas=setdiff(range(1,size(A,2)),basicas)
    N=A[:,nobasicas]
    cB=c[basicas]'
    cN=c[nobasicas]'
    #Resolvemos el problema una vez la fase 1 se completó y no hay variables artificiales en la base óptima de dicho problema
    z,B,N,basicas,nobasicas,cB,cN=ordenit(B,cB,N,cN,b,basicas,nobasicas,2);
else            
    println("El problema es infactible")
end